In [16]:
#Mount Drive and install dependencies if running in Colab
def install_dependecies():
  !pip install transformers
  !pip install pytorch-lightning

from sys import path
import os
import sys

if 'google.colab' in str(get_ipython()):
  from google.colab import drive

  root_PATH = '/content/drive/My Drive/nlp-seminar/repository'
  drive_mount_location = '/content/drive'
  module_path = root_PATH + '/src'
  
  drive.mount(drive_mount_location, force_remount=True)
  path.append(root_PATH)

  install_dependecies()
else:
  root_PATH = os.path.abspath("../../..")
  module_path = os.path.abspath(os.path.join('../../../src'))

%load_ext autoreload
%autoreload 2

if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [42]:
from data_processor import DataProcessor
from model_evaluator import ModelEvaluator
from models.BERT.bert_custom_dataset import BERTCustomDataset
from models.BERT.my_bert_model import MyBERTModel

import pandas as pd
from torch import cuda
from transformers import BertTokenizer, BertModel, BertConfig
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import transformers
import numpy as np
from sklearn import metrics
from sklearn.metrics import accuracy_score

from ipywidgets import IntProgress
from IPython.display import display

In [18]:
# Script configuration
MAX_LEN = 200
#TRAIN_BATCH_SIZE = 32
#VALID_BATCH_SIZE = 32
TEST_BATCH_SIZE = 32
#EPOCHS = 1
#LEARNING_RATE = 1e-05
model_to_use = 'bert-base-cased'

#logs_path = root_PATH + '/notebooks/deep_learning_methods/BERT/tb_logs'

#Model parameters
#model_params = {'learning_rate': LEARNING_RATE, 
#                'train_batch_size': TRAIN_BATCH_SIZE, 
#                'validation_batch_size': VALID_BATCH_SIZE,
#                'shuffle': True }

#device = 'cuda' if cuda.is_available() else 'cpu'
#gpus_to_use = [0]

#remove under-represented categories below this treshold
underrepresented_threshold = 3000
model_path = root_PATH + '/models/deep_learning/BERT/tb_logs_corrected_epoc5_backup/my_BERT_model/version_0/checkpoints/epoch=4.ckpt'

In [19]:
#Load datasets
train_df = pd.read_csv(root_PATH + '/data/train.csv')
validation_df = pd.read_csv(root_PATH + '/data/validation.csv')
test_df = pd.read_csv(root_PATH + '/data/test.csv')

#Convert topics column to list
train_df["Topic"] = train_df["Topic"].apply(eval)
validation_df["Topic"] = validation_df["Topic"].apply(eval)
test_df["Topic"] = test_df["Topic"].apply(eval)

#Generate boolean masks for our datasets
train_boolean_mask = DataProcessor.obtain_boolean_mask_from_dataset(train_df)
validation_boolean_mask = DataProcessor.obtain_boolean_mask_from_dataset(validation_df)
test_boolean_mask = DataProcessor.obtain_boolean_mask_from_dataset(test_df)

#Remove underrepresented topics
underrepresented_topics = DataProcessor.get_underrepresented_topics(train_df,underrepresented_threshold)

train_df, remaining_topics = DataProcessor.remove_topics_from_dataset(train_df,train_boolean_mask,underrepresented_topics)
validation_df, _ = DataProcessor.remove_topics_from_dataset(validation_df,validation_boolean_mask,underrepresented_topics)
test_df, _ = DataProcessor.remove_topics_from_dataset(test_df,test_boolean_mask,underrepresented_topics)

#Get boolean masks of our new dataset
train_boolean_mask = train_df.iloc[:,9:]
validation_boolean_mask = validation_df.iloc[:,9:]
test_boolean_mask = test_df.iloc[:,9:]

#Select relevant columns for training our model
train_df["list"] = train_boolean_mask.values.tolist()
validation_df["list"] = validation_boolean_mask.values.tolist()
test_df["list"] = test_boolean_mask.values.tolist()

train_df = train_df[["Conversation","list"]].reset_index()
validation_df = validation_df[["Conversation","list"]].reset_index()
test_df = test_df[["Conversation","list"]].reset_index()

In [20]:
#Instanciate my custom dataset that tokenizes the comments and generate the inputs required by BERT
tokenizer = BertTokenizer.from_pretrained(model_to_use)

training_set = BERTCustomDataset(train_df, tokenizer, MAX_LEN)
validation_set = BERTCustomDataset(validation_df, tokenizer, MAX_LEN)
testing_set = BERTCustomDataset(test_df, tokenizer, MAX_LEN)

In [21]:
model = MyBERTModel.load_from_checkpoint(model_path, hparams = {}, training_dataset=None, validation_dataset=None, labels=remaining_topics, model_to_use='bert-base-uncased')
model.eval();

In [22]:
data_loader = DataLoader(testing_set, batch_size=TEST_BATCH_SIZE, shuffle= False)

In [44]:
i = 0
predictions = np.empty([0,len(remaining_topics)])
targets = np.empty([0,len(remaining_topics)])

number_of_batches = len(data_loader)

progress_bar = IntProgress(value=0.0, min=0.0, max=number_of_batches)
display(progress_bar)

for batch in data_loader:

    batch_ids = batch['ids']
    batch_mask = batch['mask']
    batch_token_type_ids = batch['token_type_ids']
    batch_targets = batch['targets'].numpy()

    batch_outputs = model(batch_ids,batch_mask,batch_token_type_ids)
    batch_outputs = torch.sigmoid(batch_outputs).cpu().detach().numpy().tolist()
    batch_predictions = (np.array(batch_outputs) >= 0.5).astype(int)

    predictions = np.append(predictions,batch_predictions,axis=0)
    targets = np.append(targets,batch_targets,axis=0)

    progress_bar.value =+ 1
    percentage = progress_bar.value * 100 / number_of_batches
    progress_bar.description = f"{progress_bar.value}/{number_of_batches}, {percentage}%"

IntProgress(value=0, max=4912)

KeyboardInterrupt: 

In [38]:
predictions.shape
targets.shape

(160, 26)

In [39]:
print(ModelEvaluator.get_total_accuracy(targets,predictions))
print(ModelEvaluator.get_accuracy_per_label(remaining_topics,targets,predictions))

0.825
{'Satisfied users': 0.95, 'Bugs': 0.99375, 'Design & UX': 0.9875, 'Dissatisfied users': 0.98125, 'Performance': 0.98125, 'Use cases': 0.9875, 'Gaming': 1.0, 'Feature Requests': 1.0, 'Complexity': 0.99375, 'Security & Accounts': 0.9625, 'Update': 0.95, 'Pricing': 0.95, 'Camera & Photos': 0.95, 'Video': 1.0, 'Customer Support': 1.0, 'Notifications & Alerts': 1.0, 'Frequency': 1.0, 'Advertising': 1.0, 'Payment': 1.0, 'Connectivity': 0.99375, 'Devices': 1.0, 'Audio': 1.0, 'Sign Up & Login': 1.0, 'Location Services': 1.0, 'Privacy': 1.0, 'Internationalization': 1.0, 'no topic': 0.9054054054054054}
